In [1]:
import numpy as np
from fatiando.gravmag import prism, sphere
from fatiando.gravmag.eqlayer import EQLGravity
from fatiando.inversion.regularization import Smoothness2D, LCurve
from fatiando import gridder, utils, mesher
from fatiando.vis import mpl

In [ ]:
# posicao medicao

In [22]:
dx = 1.0*np.arange(10)
dx

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

In [23]:
dz = 1.0*np.arange(5,15)
dz

array([  5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.])

In [ ]:
# local dos pontos de massa 

In [29]:
dpz = -10.0*np.ones(10)

In [25]:
dpx = 1.0*np.arange(10)

In [26]:
p = 1.0*np.arange(10)

In [30]:
dado = np.zeros(dz.size)
for i in range(dx.size):
    for j in range(dpz.size):
        dado[i] = dado[i] + p[j]/np.sqrt((dpz[j]-dz[i])**2+(dpx[j]-dx[i])**2)

In [31]:
dado

array([ 2.74804941,  2.65096422,  2.54774734,  2.44177947,  2.33574522,
        2.23165328,  2.13091622,  2.03445565,  1.94280994,  1.85623143])

In [34]:
# matriz sensibilidade, G
G = np.zeros([dado.size,p.size])
for i in range(dado.size):
    for j in range(p.size):
        G[i,j] = 1./np.sqrt((dpz[j]-dz[i])**2+(dpx[j]-dx[i])**2)

In [40]:
# Conferindo G*p = dado ... obvio
print(np.dot(G,p))

print(u"Diferenca ",np.dot(G,p)-dado)

[ 2.74804941  2.65096422  2.54774734  2.44177947  2.33574522  2.23165328
  2.13091622  2.03445565  1.94280994  1.85623143]
(u'Diferen\xe7a', array([  0.00000000e+00,   0.00000000e+00,   4.44089210e-16,
         4.44089210e-16,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   2.22044605e-16,
         0.00000000e+00]))


In [42]:
dx.min()

0.0

In [49]:
# matriz sensibilidae Geq para numero arbitrario de massas equivalentes
eqx = np.arange(dx.min(),dx.max(),0.5)
eqz = -50.0*np.arange(eqx.size)
Geq = np.zeros([dado.size,eqx.size])
for i in range(dado.size):
    for j in range(eqx.size):
        Geq[i,j] = 1./np.sqrt((eqz[j]-dz[i])**2+(eqx[j]-dx[i])**2)

# Agora precisa calcular as massas equivalentes

In [56]:
Geq.shape[1]

18

In [60]:
np.eye(Geq.shape[0])

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [ ]:
+ mu*np.eye(Geq.shape[0])

In [66]:
t=np.dot(Geq, Geq.transpose()) #+ mu*np.eye(Geq.shape[0])

In [77]:
mu = 0.0001
meq = np.dot(np.dot( Geq.transpose(), np.linalg.inv(np.dot(Geq, Geq.transpose()) + mu*np.eye(Geq.shape[0])) ), dado)

In [78]:
meq

array([  6.9573401 ,  50.10106437,  28.57780659,  19.9429089 ,
        15.30820427,  12.4195753 ,  10.44737292,   9.01541013,
         7.92853152,   7.07544716,   6.38806482,   5.82238922,
         5.34873088,   4.94632941,   4.60023194,   4.29939534,
         4.03548687,   3.80210083])

In [82]:
np.abs(dado-np.dot(Geq,meq)).sum()

0.92430751794129185

In [92]:
mu = 0.000001
for i in range(3000):
    meq = np.dot(np.dot( Geq.transpose(), np.linalg.inv(np.dot(Geq, Geq.transpose()) + mu*np.eye(Geq.shape[0])) ), dado)
    err=np.abs(dado-np.dot(Geq,meq)).sum()
    if err < 0.5:
        exit
        print('massas equivalentes=',meq,' parametro mu=', mu)
    mu = mu + 1.E-5 

('massas equivalentes=', array([  4.34987554,  98.18327401,  20.55744801,   5.44510968,
         0.67436287,  -1.18121338,  -1.97030993,  -2.30530048,
        -2.42789027,  -2.44539246,  -2.40956304,  -2.346884  ,
        -2.27147822,  -2.1910794 ,  -2.10997307,  -2.03052145,
        -1.95398781,  -1.88099725]), ' parametro mu=', 1e-06)


In [91]:
err

5.873787507756818